# Langchain Foundations

This notebook introduces the foundations of **LangChain**, a framework that simplifies working with Large Language Models (LLMs). LangChain provides:
- Abstraction layers to streamline interactions with various LLMs.
- Standardized interfaces for working with different types of AI models.
- Tools and utilities for developing complex AI applications.



This notebook is divided into two sections:
1. Step by Step: configuration, templates, and basic chaining
2. Combined Output: highlighting step-wise vs. chain execution


## 1. Step by Step

In [ ]:
# install notebook package dependencies
!pip install langchain==0.3.2
!pip install langchain-openai==0.2.0

In [ ]:
import getpass
import os

# set openai api key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAi Key:")

In [3]:
from langchain_openai import ChatOpenAI

# initialize the model
model = ChatOpenAI(model="gpt-4o-mini")

LangChain provides message templates to structure interactions:

SystemMessage defines instructions or context.
HumanMessage represents user input.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# system and human message templates
messages = [
    SystemMessage(content="You are an expert financial analyst."),
    HumanMessage(content="How much has NVIDIA's revenue grown over time?"),
]

# print message
print("message template")
print(messages)

# e.g., system message
print("\nparse message template")
print(messages[0].type)
print(messages[0].content)

# invoke the model with messages
print("\ninvoke model")
print(model.invoke(messages))

In [5]:
from langchain_core.output_parsers import StrOutputParser

# initialize prebuilt output parser
parser = StrOutputParser()

Using model.invoke(messages), we pass in our defined messages and receive the model's response - simplifying model interaction

In [6]:
# invoke model
result = model.invoke(messages)

Output parsers help structure and format responses. Here, the StrOutputParser converts model outputs into standardized string formats.

In [ ]:
# pass through string output parser
parser.invoke(result)

Invoke a predefined chain, which processes messages and outputs a structured response.

In [8]:
# chain together model and parser
chain = model | parser

In [ ]:
# output is now a string
chain.invoke(messages)

In [10]:
from langchain_core.prompts import ChatPromptTemplate

We set up a system message template that includes a {company} variable as a placeholder. This enables dynamic updates to the template based on specific user input.

In [11]:
# system template with placeholder language variable
system_template = "You are an expert financial analyst that specializes in analyzing {company}."

We construct a Chat Prompt Template by combining the system message with a user message placeholder {text}.

In [ ]:
# construct chat prompt template with system template and user message placeholder
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# print the prompt template
print(prompt_template.messages)

Using .invoke(), we pass specific values for company and text.

In [ ]:
# pass in variables
result = prompt_template.invoke({"company": "NVIDIA", "text": "How much has NVIDIA's revenue grown over time?"})

result

The to_messages() method converts the result back into a list of SystemMessage and HumanMessage objects.

In [ ]:
# same messages structure as directly using system and human message templates
result.to_messages()

LangChain enables us to combine templates, models, and parsers into a single chain object.

In [15]:
# chain all together
chain = prompt_template | model | parser

We invoke the full chain with company and text input, executing each component (prompt, model, and parser) in sequence. The output is a complete, processed response ready for display or further processing.

In [ ]:
# invoke the chain with input variables
chain.invoke({"company": "NVIDIA", "text": "How much has NVIDIA's revenue grown over time?"})

## 2. Combined Output

Here, we compare step-wise execution (invoking each element individually) with chain execution (invoking the chain as a single unit).

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# define the model
model = ChatOpenAI(model="gpt-4o-mini")
# define the prompt template
system_template = "You are an expert financial analyst that specializes in analyzing {company}."
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
# define the output parser
parser = StrOutputParser()
# step wise result
result = prompt_template.invoke({"company": "NVIDIA", "text": "How much has NVIDIA's revenue grown over time?"})
result = model.invoke(result)
result = parser.invoke(result)
print("Step-wise:")
print(result)
# chain result
print("\nChain:")
chain = prompt_template | model | parser
result = chain.invoke({"company": "NVIDIA", "text": "How much has NVIDIA's revenue grown over time?"})
print(result)